<a href="https://colab.research.google.com/github/hornetio/sales_prediction/blob/main/experiments/many_alghoritms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn xgboost lightgbm catboost statsmodels matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" an

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
test_df=pd.read_csv('test.csv')
train_df=pd.read_csv('train.csv')
add_df=pd.read_csv('store.csv')

<ipython-input-3-18c1b6495887>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df=pd.read_csv('train.csv')


In [ ]:
test_df.fillna(0,inplace=True)

In [ ]:
add_df.isna().sum()

,0
Store,0
StoreType,0
Assortment,0
CompetitionDistance,3
CompetitionOpenSinceMonth,354
CompetitionOpenSinceYear,354
Promo2,0
Promo2SinceWeek,544
Promo2SinceYear,544
PromoInterval,544


In [ ]:
add_df.loc[add_df['Promo2'] == 0, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']] = 0

In [ ]:
add_df['CompetitionDistance'].fillna(add_df['CompetitionDistance'].mean(), inplace=True)
add_df['CompetitionOpenSinceMonth'].fillna(int(add_df['CompetitionOpenSinceMonth'].mean()), inplace=True)
add_df['CompetitionOpenSinceYear'].fillna(int(add_df['CompetitionOpenSinceYear'].mean()), inplace=True)

In [ ]:
data = pd.merge(train_df, add_df, on='Store')
data = data[data['Open'] == 1]
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['WeekOfYear'] = data['Date'].dt.isocalendar().week


data['CompetitionDistance'].fillna(data['CompetitionDistance'].max(), inplace=True)
data.fillna(0, inplace=True)





data.dropna(inplace=True)


data=data.drop('Date',axis=1)
target = 'Sales'


data = pd.get_dummies(data, columns=['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval'])

train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

In [ ]:
train_data.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'Year', 'Month', 'Day', 'WeekOfYear', 'Sales_MA_7',
       'Sales_MA_30', 'StateHoliday_0', 'StateHoliday_0', 'StateHoliday_a',
       'StateHoliday_b', 'StateHoliday_c', 'StoreType_a', 'StoreType_b',
       'StoreType_c', 'StoreType_d', 'Assortment_a', 'Assortment_b',
       'Assortment_c', 'PromoInterval_0', 'PromoInterval_Feb,May,Aug,Nov',
       'PromoInterval_Jan,Apr,Jul,Oct', 'PromoInterval_Mar,Jun,Sept,Dec'],
      dtype='object')

In [ ]:
print(train_data[features].dtypes)

KeyError: "['PromoInterval_Feb,May,Aug,Nov', 'PromoInterval_Jan,Apr,Jul,Oct', 'PromoInterval_Mar,Jun,Sept,Dec'] not in index"

In [ ]:
train_data = train_data.loc[:, ~train_data.columns.duplicated()]
test_data = test_data.loc[:, ~test_data.columns.duplicated()]

# Конвертация булевых столбцов в int
bool_columns = train_data.select_dtypes(include=['bool']).columns
train_data[bool_columns] = train_data[bool_columns].astype(int)
test_data[bool_columns] = test_data[bool_columns].astype(int)

# Проверка на дублирующиеся столбцы
print(train_data.columns.duplicated().sum())  # Должно быть 0

0


<ipython-input-52-fe26df4d97e9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[bool_columns] = train_data[bool_columns].astype(int)
<ipython-input-52-fe26df4d97e9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[bool_columns] = test_data[bool_columns].astype(int)


In [ ]:
train_data = train_data.groupby(axis=1, level=0).sum()
test_data = test_data.groupby(axis=1, level=0).sum()

<ipython-input-53-fdc716830ef6>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  train_data = train_data.groupby(axis=1, level=0).sum()
<ipython-input-53-fdc716830ef6>:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  test_data = test_data.groupby(axis=1, level=0).sum()


In [ ]:
train_data.columns = train_data.columns.str.replace('[^\w\s]', '', regex=True)
test_data.columns=test_data.columns.str.replace('[^\w\s]','',regex=True)

In [ ]:
X_train=train_data.drop('Sales',axis=1)
y_train=train_data['Sales']
X_test=test_data.drop('Sales',axis=1)
y_test=test_data['Sales']

In [ ]:
lgb_model = LGBMRegressor(n_estimators=200, learning_rate=0.1)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077134 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1491
[LightGBM] [Info] Number of data points in the train set: 675490, number of used features: 32
[LightGBM] [Info] Start training from score 6914.695836


LGBMRegressor(n_estimators=200)

In [ ]:
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.1)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, silent=True, has_time=True)
cat_model.fit(X_train, y_train)



In [ ]:
hw_model = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12)
hw_model_fit = hw_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


TypeError: smoothing_level must be float_like (float or np.inexact) or None

In [ ]:
test_data.columns

Index(['Assortment_a', 'Assortment_b', 'Assortment_c', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Customers',
       'Day', 'DayOfWeek', 'Month', 'Open', 'Promo', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval_0',
       'PromoInterval_FebMayAugNov', 'PromoInterval_JanAprJulOct',
       'PromoInterval_MarJunSeptDec', 'Sales', 'Sales_MA_30', 'Sales_MA_7',
       'SchoolHoliday', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c', 'Store', 'StoreType_a', 'StoreType_b', 'StoreType_c',
       'StoreType_d', 'WeekOfYear', 'Year'],
      dtype='object')

In [ ]:
# Предсказания
xgb_preds = xgb_model.predict(X_test)
lgb_preds = lgb_model.predict(X_test)
cat_preds = cat_model.predict(X_test)
hw_preds = hw_model_fit.forecast(steps=len(X_test))

# Вычисление метрик
def evaluate_model(true, predicted, model_name):
    rmse = np.sqrt(mean_squared_error(true, predicted))
    mae = mean_absolute_error(true, predicted)
    print(f'{model_name} - RMSE: {rmse}, MAE: {mae}')
    return rmse, mae

xgb_metrics = evaluate_model(y_test, xgb_preds, "XGBoost")
lgb_metrics = evaluate_model(y_test, lgb_preds, "LightGBM")
cat_metrics = evaluate_model(y_test, cat_preds, "CatBoost")
hw_metrics = evaluate_model(y_test, hw_preds, "Holt-Winters")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


XGBoost - RMSE: 696.4028673128937, MAE: 495.6576953961196
LightGBM - RMSE: 705.103795403591, MAE: 505.85957190726504
CatBoost - RMSE: 597.1397800187974, MAE: 415.4859609598854
Holt-Winters - RMSE: 3434.822639817313, MAE: 2821.65781553911


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)

sequence_length = 30
train_generator = TimeseriesGenerator(scaled_train, y_train.values, length=sequence_length, batch_size=32)
test_generator = TimeseriesGenerator(scaled_test, y_test.values, length=sequence_length, batch_size=32)


In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)


In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)


In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, scaled_train.shape[1])))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=1))

lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(train_generator, epochs=10, verbose=1)

# Предсказания для LSTM
lstm_preds = lstm_model.predict(test_generator)
lstm_preds = scaler.inverse_transform(lstm_preds)  # Обратное масштабирование
lstm_preds = lstm_preds.flatten()  # Преобразование в одномерный массив


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    1/21109 ━━━━━━━━━━━━━━━━━━━━ 23:56:34 4s/step - loss: 24438548.0000

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21109/21109 ━━━━━━━━━━━━━━━━━━━━ 734s 35ms/step - loss: 54052840.0000
Epoch 2/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 740s 35ms/step - loss: 41127112.0000
Epoch 3/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 743s 35ms/step - loss: 30882134.0000
Epoch 4/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 722s 34ms/step - loss: 22892794.0000
Epoch 5/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 743s 34ms/step - loss: 17054950.0000
Epoch 6/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 738s 35ms/step - loss: 13429830.0000
Epoch 7/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 780s 37ms/step - loss: 11099760.0000
Epoch 8/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 754s 35ms/step - loss: 10280800.0000
Epoch 9/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 771s 37ms/step - loss: 10062169.0000
Epoch 10/10
21109/21109 ━━━━━━━━━━━━━━━━━━━━ 867s 41ms/step - loss: 9925004.0000
5277/5277 ━━━━━━━━━━━━━━━━━━━━ 75s 14ms/step


ValueError: non-broadcastable output operand with shape (168843,1) doesn't match the broadcast shape (168843,33)

In [ ]:
# Оценка моделей
rf_metrics = evaluate_model(test_data[target], rf_preds, "Random Forest")
lr_metrics = evaluate_model(test_data[target], lr_preds, "Linear Regression")
lstm_metrics = evaluate_model(test_data[target][sequence_length:], lstm_preds, "LSTM")

# # Визуализация результатов для всех моделей
# plt.figure(figsize=(14,7))
# plt.plot(test_data['Date'], test_data['Sales'], label='Actual Sales', color='black')
# plt.plot(test_data['Date'], xgb_preds, label='XGBoost Predictions', color='green')
# plt.plot(test_data['Date'], lgb_preds, label='LightGBM Predictions', color='blue')
# plt.plot(test_data['Date'], cat_preds, label='CatBoost Predictions', color='red')
# plt.plot(test_data['Date'], hw_preds, label='Holt-Winters Predictions', color='orange')
# plt.plot(test_data['Date'], rf_preds, label='Random Forest Predictions', color='purple')
# plt.plot(test_data['Date'], lr_preds, label='Linear Regression Predictions', color='brown')
# plt.plot(test_data['Date'][sequence_length:], lstm_preds, label='LSTM Predictions', color='magenta')
# plt.legend()
# plt.show()


Random Forest - RMSE: 579.3976802122856, MAE: 393.6878104848021
Linear Regression - RMSE: 1236.5876214358427, MAE: 897.3663709871284
LSTM - RMSE: 3082.866284596447, MAE: 2227.1859684973847
